In [2]:
import os
import cv2
import random
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader, random_split

In [3]:
CFG = {'batch_size': 32,
       'learning_rate': 1e-4,
       'seed':24,
       'epochs': 100   
}

In [4]:
if torch.cuda.is_available() == True:
       device = 'cuda:0'
       print('현재 가상환경 GPU 사용 가능 상태')
else:
       device = 'cpu'
       print('GPU 사용 불가능 상태')

GPU 사용 불가능 상태


In [3]:
def seed_everything(seed: int = 24):
       random.seed(seed)
       np.random.seed(seed)
       # torch.cuda.manual_seed(seed)
       # torch.backends.cudnn.deterministic = True
seed_everything()

In [4]:
tot_actions = 5
actions_name = 'backward', 'sit', 'slide', 'swing', 'walk'
min_data_len = 45

In [5]:
actions_csv_dir = '../csv_1031/'
dataset = []

label_mapping = {'backward': 0,
                 'sit': 1,
                 'slide': 2,
                 'swing': 3,
                 'walk' : 4
                 }

def map_action_to_label(csv_name):
       for action, label in label_mapping.items():
              if action in csv_name.split('_')[0]:
                     return label
       return -1


for action_csv in os.listdir(actions_csv_dir):
       action_df = pd.read_csv(os.path.join(actions_csv_dir, action_csv))
       
       label = map_action_to_label(action_csv)
       if label != -1:
              for idx in range(0, len(action_df), int(min_data_len / 2)):
                     seq_df = action_df[idx: idx + min_data_len] #길이만큼 데이터 자른 것(즉 length 만큼의 프레임)
                     if len(seq_df) == min_data_len: # 딱 length에 개수 맞춰서 끊어서 넣으려고
                            dataset.append({'key': label, 'value': seq_df}) # key에 slide, value에는 묶음 프레임 만큼이 담기겠네
       #최종적으로 dataset에는 행동별로 dictionary 가 만들어져 들어간다.

In [6]:
print(dataset[0]['value'].columns) # z축 까지 99 (33 * 3)차원

Index(['Nose_x', 'Nose_y', 'Nose_z', 'LEye_in_x', 'LEye_in_y', 'LEye_in_z',
       'LEye_x', 'LEye_y', 'LEye_z', 'LEye_out_x', 'LEye_out_y', 'LEye_out_z',
       'REye_in_x', 'REye_in_y', 'REye_in_z', 'REye_x', 'REye_y', 'REye_z',
       'REye_out_x', 'REye_out_y', 'REye_out_z', 'LEar_x', 'LEar_y', 'LEar_z',
       'REar_x', 'REar_y', 'REar_z', 'LMouth_x', 'LMouth_y', 'LMouth_z',
       'RMouth_x', 'RMouth_y', 'RMouth_z', 'LShoulder_x', 'LShoulder_y',
       'LShoulder_z', 'RShoulder_x', 'RShoulder_y', 'RShoulder_z', 'LElbow_x',
       'LElbow_y', 'LElbow_z', 'RElbow_x', 'RElbow_y', 'RElbow_z', 'LWrist_x',
       'LWrist_y', 'LWrist_z', 'RWrist_x', 'RWrist_y', 'RWrist_z', 'LPinky_x',
       'LPinky_y', 'LPinky_z', 'RPinky_x', 'RPinky_y', 'RPinky_z', 'LIndex_x',
       'LIndex_y', 'LIndex_z', 'RIndex_x', 'RIndex_y', 'RIndex_z', 'LThumb_x',
       'LThumb_y', 'LThumb_z', 'RThumb_x', 'RThumb_y', 'RThumb_z', 'LHip_x',
       'LHip_y', 'LHip_z', 'RHip_x', 'RHip_y', 'RHip_z', 'LKnee_x', 'LKn

In [7]:
class MyDataset(Dataset):
       def __init__(self, dataset): #모든 행동을 통합한 df가 들어가야함
              self.x = []
              self.y = []
              for dic in dataset:
                     self.y.append(dic['key']) #key 값에는 actions 들어감
                     self.x.append(dic['value']) #action마다의 data 들어감

       def __getitem__(self, index): #index는 행동의 index
              data = self.x[index] # x에는 꺼내 쓸 (행동마다 45개 묶음프레임)의 데이터
              label = self.y[index]
              return torch.Tensor(np.array(data)), torch.tensor(np.array(int(label)))

       def __len__(self):
              return len(self.x)


In [8]:
train_test_val_ratio = [0.8, 0.1, 0.1]
print(len(dataset))
train_len = int(len(dataset) * train_test_val_ratio[0])
val_len = int(len(dataset) * train_test_val_ratio[1])
test_len = len(dataset) - train_len - val_len
print('{}, {}, {}'.format(train_len, val_len, test_len))

664
531, 66, 67


In [10]:
train_dataset = MyDataset(dataset)
train_data, valid_data, test_data = random_split(train_dataset, [train_len, val_len, test_len])

train_loader = DataLoader(train_data, batch_size=CFG['batch_size'])
val_loader = DataLoader(valid_data, batch_size=CFG['batch_size'])
test_loader = DataLoader(test_data, batch_size=CFG['batch_size'])

In [11]:
class Model1(nn.Module):
       def __init__(self):
              super(Model1, self).__init__()
              self.lstm1 = nn.LSTM(input_size=99, hidden_size=128, num_layers=1, batch_first=True) #input은  45 * 3(x, y z)
              self.lstm2 = nn.LSTM(input_size=128, hidden_size=256, num_layers=1, batch_first=True)
              self.lstm3 = nn.LSTM(input_size=256, hidden_size=512, num_layers=1, batch_first=True)
              self.dropout1 = nn.Dropout(0, 1)
              self.lstm4 = nn.LSTM(input_size=512, hidden_size=256, num_layers=1, batch_first=True)
              self.lstm5 = nn.LSTM(input_size=256, hidden_size=128, num_layers=1, batch_first=True)
              self.lstm6 = nn.LSTM(input_size=128, hidden_size=64, num_layers=1, batch_first=True)
              self.dropout2 = nn.Dropout(0, 1)
              self.lstm7 = nn.LSTM(input_size=64, hidden_size=32, num_layers=1, batch_first=True)
              self.fc = nn.Linear(32, 5) #분류할 클래스 5가지

       def forward(self, x):
              x, _ = self.lstm1(x)
              x, _ = self.lstm2(x)
              x, _ = self.lstm3(x)
              x = self.dropout1(x)
              x, _ = self.lstm4(x)
              x, _ = self.lstm5(x)
              x, _ = self.lstm6(x)
              x = self.dropout2(x)
              x, _ = self.lstm7(x)
              x = self.fc(x[:, -1, :])
              return x

In [11]:
# Model 2
from torch.autograd import Variable


class ConvLSTMCell(nn.Module):
       def __init__(self, input_channels, hidden_channels, kernel_size):
              super(ConvLSTMCell, self).__init__()

              assert hidden_channels % 2 == 0

              self.input_channels = input_channels
              self.hidden_channels = hidden_channels
              self.kernel_size = kernel_size
              self.num_features = 5

              self.padding = int((kernel_size - 1) / 2)

              self.Wxi = nn.Conv2d(self.input_channels, self.hidden_channels, self.kernel_size, 1, self.padding, bias=True)
              self.Whi = nn.Conv2d(self.hidden_channels, self.hidden_channels, self.kernel_size, 1, self.padding, bias=False)
              self.Wxf = nn.Conv2d(self.input_channels, self.hidden_channels, self.kernel_size, 1, self.padding, bias=True)
              self.Whf = nn.Conv2d(self.hidden_channels, self.hidden_channels, self.kernel_size, 1, self.padding, bias=False)
              self.Wxc = nn.Conv2d(self.input_channels, self.hidden_channels, self.kernel_size, 1, self.padding, bias=True)
              self.Whc = nn.Conv2d(self.hidden_channels, self.hidden_channels, self.kernel_size, 1, self.padding, bias=False)
              self.Wxo = nn.Conv2d(self.input_channels, self.hidden_channels, self.kernel_size, 1, self.padding, bias=True)
              self.Who = nn.Conv2d(self.hidden_channels, self.hidden_channels, self.kernel_size, 1, self.padding, bias=False)

              self.Wci = None
              self.Wcf = None
              self.Wco = None

       def forward(self, x, h, c):
              ci = torch.sigmoid(self.Wxi(x) + self.Whi(h) + c * self.Wci)
              cf = torch.sigmoid(self.Wxf(x) + self.Whf(h) + c * self.Wcf)
              cc = cf * c + ci * torch.tanh(self.Wxc(x) + self.Whc(h))
              co = torch.sigmoid(self.Wxo(x) + self.Who(h) + cc * self.Wco)
              ch = co * torch.tanh(cc)
              return ch, cc

       def init_hidden(self, batch_size, hidden, shape):
              if self.Wci is None:
                     self.Wci = nn.Parameter(torch.zeros(1, hidden, shape[0], shape[1])).cuda()
                     self.Wcf = nn.Parameter(torch.zeros(1, hidden, shape[0], shape[1])).cuda()
                     self.Wco = nn.Parameter(torch.zeros(1, hidden, shape[0], shape[1])).cuda()
              else:
                     assert shape[0] == self.Wci.size()[2], 'Input Height Mismatched!'
                     assert shape[1] == self.Wci.size()[3], 'Input Width Mismatched!'
              return (Variable(torch.zeros(batch_size, hidden, shape[0], shape[1])).cuda(),
                      Variable(torch.zeros(batch_size, hidden, shape[0], shape[1])).cuda())


class ConvLSTM(nn.Module):
       # input_channels corresponds to the first input feature map
       # hidden state is a list of succeeding lstm layers.
       def __init__(self, input_channels, hidden_channels, kernel_size, step=1, effective_step=[1]):
              super(ConvLSTM, self).__init__()
              self.input_channels = [input_channels] + hidden_channels
              self.hidden_channels = hidden_channels
              self.kernel_size = kernel_size
              self.num_layers = len(hidden_channels)
              self.step = step
              self.effective_step = effective_step
              self._all_layers = []
              for i in range(self.num_layers):
                     name = 'cell{}'.format(i)
                     cell = ConvLSTMCell(self.input_channels[i], self.hidden_channels[i], self.kernel_size)
                     setattr(self, name, cell)
                     self._all_layers.append(cell)

       def forward(self, input):
              internal_state = []
              outputs = []
              for step in range(self.step):
                     x = input
                     for i in range(self.num_layers):
                            # all cells are initialized in the first step
                            name = 'cell{}'.format(i)
                            if step == 0:
                                   bsize, _, height, width = x.size()
                                   (h, c) = getattr(self, name).init_hidden(batch_size=bsize, hidden=self.hidden_channels[i],
                                                                            shape=(height, width))
                                   internal_state.append((h, c))

                            # do forward
                            (h, c) = internal_state[i]
                            x, new_c = getattr(self, name)(x, h, c)
                            internal_state[i] = (x, new_c)
                     # only record effective steps
                     if step in self.effective_step:
                            outputs.append(x)

              return outputs, (x, new_c)


In [12]:
# gradient check
convlstm = ConvLSTM(input_channels=512, hidden_channels=[128, 64, 64, 32, 32], kernel_size=3, step=5,
                    effective_step=[4]).cuda()
loss_fn = torch.nn.MSELoss()

input = Variable(torch.randn(1, 512, 64, 32)).cuda()
target = Variable(torch.randn(1, 32, 64, 32)).double().cuda()

output = convlstm(input)
output = output[0][0].double()
res = torch.autograd.gradcheck(loss_fn, (output, target), eps=1e-6, raise_exception=True)

print(res)

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
# Model 3
class CGRU_cell(nn.Module):
       """
       ConvGRU Cell
       """
       def __init__(self, shape, input_channels, filter_size, num_features):
              super(CGRU_cell, self).__init__()
              self.shape = shape
              self.input_channels = input_channels
              # kernel_size of input_to_state equals state_to_state
              self.filter_size = filter_size
              self.num_features = num_features
              self.padding = (filter_size - 1) // 2
              self.conv1 = nn.Sequential(
                     nn.Conv2d(self.input_channels + self.num_features,
                               2 * self.num_features, self.filter_size, 1,
                               self.padding),
                     nn.GroupNorm(2 * self.num_features // 32, 2 * self.num_features))
              self.conv2 = nn.Sequential(
                     nn.Conv2d(self.input_channels + self.num_features,
                               self.num_features, self.filter_size, 1, self.padding),
                     nn.GroupNorm(self.num_features // 32, self.num_features))

       def forward(self, inputs=None, hidden_state=None, seq_len=10):
              # seq_len=10 for moving_mnist
              if hidden_state is None:
                     htprev = torch.zeros(inputs.size(1), self.num_features,
                                          self.shape[0], self.shape[1]).cuda()
              else:
                     htprev = hidden_state
              output_inner = []
              for index in range(seq_len):
                     if inputs is None:
                            x = torch.zeros(htprev.size(0), self.input_channels,
                                            self.shape[0], self.shape[1]).cuda()
                     else:
                            x = inputs[index, ...]

                     combined_1 = torch.cat((x, htprev), 1)  # X_t + H_t-1
                     gates = self.conv1(combined_1)  # W * (X_t + H_t-1)

                     zgate, rgate = torch.split(gates, self.num_features, dim=1)
                     # zgate, rgate = gates.chunk(2, 1)
                     z = torch.sigmoid(zgate)
                     r = torch.sigmoid(rgate)

                     combined_2 = torch.cat((x, r * htprev),
                                            1)  # h' = tanh(W*(x+r*H_t-1))
                     ht = self.conv2(combined_2)
                     ht = torch.tanh(ht)
                     htnext = (1 - z) * htprev + z * ht
                     output_inner.append(htnext)
                     htprev = htnext
              return torch.stack(output_inner), htnext


In [ ]:
# gradient check
cgru = CGRU_cell((32, 45, ), input_channels=512 ) 
loss_fn = torch.nn.MSELoss()

input = Variable(torch.randn(1, 512, 64, 32)).cuda()
target = Variable(torch.randn(1, 32, 64, 32)).double().cuda()

output = convlstm(input)
output = output[0][0].double()
res = torch.autograd.gradcheck(loss_fn, (output, target), eps=1e-6, raise_exception=True)

print(res)

In [12]:
from dev.pytorchtools import EarlyStopping
early_stopping = EarlyStopping(patience = 30, verbose = True)

num_epochs = 150
learning_rate = 1e-3
hidden_size = 512
num_layers = 2
# quantile = 0.3


train_losses = []
train_accs = []
valid_losses = []
valid_accs = []

val_acc_max = 0.75
val_loss_min = 0.25

lstm = Model1()

# criterion = quantile_loss    # mean-squared error for regression
criterion = torch.nn.NLLLoss()   # mean-squared error for regression
# optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate,weight_decay= 1e-5)
criterion = criterion.cuda()

m = nn.Softmax(dim = 1)

optimizer = torch.optim.SGD(lstm.parameters(), lr=learning_rate,momentum=0.9)

# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,  patience= 60, 
#           factor =0.1 ,min_lr=1e-6, eps=1e-08)

# scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,700,900,1000], gamma=0.1)

for epoch in range(num_epochs+1):
       losses = []
       acc_list = []
       for i, (input, target) in enumerate(train_loader):

              lstm.train()
              outputs = lstm(input.to(device))
              optimizer.zero_grad()
              
              target = target.long()
              loss = criterion(m(outputs), target.type(torch.long).to(device))
              loss.backward()
              optimizer.step()
              losses.append(loss.item())
              preds = torch.argmax(outputs, dim=1)
              batch_acc = (preds == target).float().mean()
              acc_list.append(batch_acc.item())
           

       train_losses.append(np.mean(np.array(losses)))
     
       losses = []
       acc_list = []
       for i, (input, target) in enumerate(val_loader):
              lstm.eval()
              with torch.no_grad():
                     valid = lstm(input.to(device))
                     target = target.long()
                     val_loss = criterion(m(valid), target.type(torch.long).to(device))
                     preds = torch.argmax(valid, dim=1)
                     batch_acc = (preds == target).float().mean()
                     losses.append(val_loss.item())
                     acc_list.append(batch_acc.item())
                     if val_acc_max < batch_acc.item():
                            val_acc_max = batch_acc.item()
                            print('Model saved, model val acc: ', batch_acc.item())
                            
                            torch.save(lstm, fr'C:\Users\USER\Desktop\19rne\2023-RnE-main\save_by_acc\model_{epoch}.pth')

                     if val_loss_min > val_loss:
                            val_loss_min = val_loss
                            torch.save(lstm, fr'C:\Users\USER\Desktop\19rne\2023-RnE-main\save_by_loss\model_{epoch}.pth')
                     

       valid_losses.append(np.mean(np.array(losses)))
       valid_accs.append(np.mean(np.array(losses)))



       if epoch % 10 == 0:
              # print(criterion1(outputs, y_train.to(device),quantile))

              print("Epoch: %d, loss: %1.5f valid loss:  %1.5f lr: %1.5f " %(epoch, train_losses[-1],valid_losses[-1],
                                                                             optimizer.param_groups[0]["lr"]))
       
       
       # model.load_state_dict(torch.load(SAVEPATH+'model_weight.pth'))

       # early_stopping는 validation loss가 감소하였는지 확인이 필요하며,
       # 만약 감소하였을경우 현제 모델을 checkpoint로 만든다.
       early_stopping(round(valid_losses[-1],5), lstm)

       if early_stopping.early_stop:
              print("Epoch: %d, loss: %1.5f valid loss:  %1.5f lr: %1.5f " %(epoch, train_losses[-1],valid_losses[-1],
                                                                             optimizer.param_groups[0]["lr"]))
              break
           


Epoch: 0, loss: -0.19235 valid loss:  -0.19264 lr: 0.00100 
Validation loss decreased (inf --> -0.192640).  Saving model ...
Validation loss decreased (-0.192640 --> -0.193880).  Saving model ...
Validation loss decreased (-0.193880 --> -0.195230).  Saving model ...
Validation loss decreased (-0.195230 --> -0.196600).  Saving model ...
Validation loss decreased (-0.196600 --> -0.198010).  Saving model ...
Validation loss decreased (-0.198010 --> -0.199440).  Saving model ...
Validation loss decreased (-0.199440 --> -0.200890).  Saving model ...
Validation loss decreased (-0.200890 --> -0.202360).  Saving model ...
Validation loss decreased (-0.202360 --> -0.203870).  Saving model ...
Validation loss decreased (-0.203870 --> -0.205400).  Saving model ...
Epoch: 10, loss: -0.20832 valid loss:  -0.20695 lr: 0.00100 
Validation loss decreased (-0.205400 --> -0.206950).  Saving model ...
Validation loss decreased (-0.206950 --> -0.208540).  Saving model ...
Validation loss decreased (-0.208

In [15]:
# Plotting the training and validation loss
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss')
plt.plot(valid_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss over Epochs')
plt.legend()
plt.show()

# Plotting the training and validation accuracy
plt.figure(figsize=(10, 5))
plt.plot(train_accs, label='Training Accuracy')
plt.plot(valid_accs, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy over Epochs')
plt.legend()
plt.show()


OrderedDict([('lstm1.weight_ih_l0', tensor([[ 0.0106,  0.0242, -0.0078,  ...,  0.0666, -0.0148, -0.0841],
        [-0.0231, -0.0550,  0.0528,  ...,  0.0713, -0.0040, -0.0330],
        [ 0.0231,  0.0690,  0.0860,  ...,  0.0452,  0.0325,  0.0387],
        ...,
        [ 0.0411, -0.0150,  0.0160,  ...,  0.0471, -0.0437, -0.0269],
        [ 0.0147, -0.0360, -0.0769,  ..., -0.0176, -0.0588, -0.0270],
        [-0.0545,  0.0651,  0.0564,  ..., -0.0187,  0.0805, -0.0175]])), ('lstm1.weight_hh_l0', tensor([[ 0.0696, -0.0242,  0.0037,  ...,  0.0748, -0.0340,  0.0232],
        [-0.0830,  0.0406, -0.0691,  ..., -0.0431, -0.0609, -0.0500],
        [ 0.0299,  0.0226,  0.0513,  ...,  0.0128,  0.0134,  0.0247],
        ...,
        [-0.0672,  0.0084, -0.0638,  ..., -0.0733, -0.0641, -0.0737],
        [ 0.0594,  0.0357,  0.0599,  ...,  0.0397, -0.0346, -0.0428],
        [-0.0431,  0.0424, -0.0633,  ..., -0.0643,  0.0663,  0.0109]])), ('lstm1.bias_ih_l0', tensor([-0.0511, -0.0311, -0.0554,  0.0446,  0.0

RuntimeError: a Tensor with 160 elements cannot be converted to Scalar